In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.auto import tqdm
import os
from PIL import Image
import sys

In [ ]:
def is_valid_image(filepath):
    try:
        with Image.open(filepath) as img:
            img.verify()
        return True
    except:
        print(f"Corrupted image found: {filepath}", file=sys.stderr)
        return False

class FilteredImageFolder(datasets.ImageFolder):
    def __init__(self, root, transform=None):
        super(FilteredImageFolder, self).__init__(root=root, transform=transform)
        # Filter out corrupt images
        valid_images = []
        for item in self.imgs:
            if is_valid_image(item[0]):
                valid_images.append(item)
        self.imgs = valid_images
        self.samples = valid_images

In [ ]:
img_width, img_height = 150, 150
batch_size = 32
num_epochs = 10

# Data augmentation and normalization for training
# Just normalization for validation
train_transform = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.RandomRotation(40),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
print("Loading and validating training dataset...")
train_dataset = FilteredImageFolder(
    root='dataset/train',
    transform=train_transform
)

print("Loading and validating validation dataset...")
val_dataset = FilteredImageFolder(
    root='dataset/validation',
    transform=val_transform
)

print("Training classes:", train_dataset.classes)
print("Validation classes:", val_dataset.classes)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)


In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()
        self.features = nn.Sequential(
            # First convolutional block
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Second convolutional block
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Third convolutional block
            nn.Conv2d(64, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Fourth convolutional block
            nn.Conv2d(128, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avg_pool(x)
        x = self.classifier(x)
        return x

# Initialize model, loss function, and optimizer
model = CNNClassifier(num_classes=len(train_dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Print model summary
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nModel Parameters: {count_parameters(model):,}")

In [ ]:
import matplotlib.pyplot as plt

# Lists to store losses
train_losses = []
val_losses = []

# Modified training loop to track losses
best_val_acc = 0.0
for epoch in range(num_epochs):
    print(f'\nEpoch [{epoch+1}/{num_epochs}]')
    
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    pbar = tqdm(total=len(train_loader), desc=f'Training')
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs, labels
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        grad_norms = {}
        for name, param in model.named_parameters():
            if param.grad is not None:
                grad_norms[name] = param.grad.norm().item()
        
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        
        avg_loss = train_loss / (i + 1)
        acc = 100. * train_correct / train_total
        pbar.set_description(f'Train | Loss: {avg_loss:.4f} | Acc: {acc:.2f}%')
        pbar.update(1)
    
    pbar.close()
    
    # Store average training loss
    train_losses.append(train_loss / len(train_loader))
    
    print("\nGradient Norms:")
    for name, norm in grad_norms.items():
        print(f"{name}: {norm:.4f}")
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    pbar = tqdm(total=len(val_loader), desc=f'Validation')
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs, labels = inputs, labels
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
            
            avg_loss = val_loss / (i + 1)
            acc = 100. * val_correct / val_total
            pbar.set_description(f'Val | Loss: {avg_loss:.4f} | Acc: {acc:.2f}%')
            pbar.update(1)
    
    pbar.close()
    
    # Store average validation loss
    val_losses.append(val_loss / len(val_loader))
    
    train_acc = 100. * train_correct / train_total
    val_acc = 100. * val_correct / val_total
    
    print(f'\nEpoch Summary:')
    print(f'Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc:.2f}%')
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'catsanddogs_classifier.pth')
        print(f'New best model saved! (Validation Accuracy: {val_acc:.2f}%)')

# Plotting code
plt.figure(figsize=(8, 6))
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss', marker='o')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('loss_plot_cnn.png')
plt.show()

In [ ]:
torch.save(model, 'catsanddogs_classifier_full.pth')

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss', marker='o')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('loss_plot_cnn.png')
plt.show()  # Optional: Display the plot

In [ ]:
def predict_image(image_path, model, transform, device, classes=['cats', 'dogs']):
    from PIL import Image
    import torch
    try:
        # Set model to evaluation mode
        model.eval()
        # Open and preprocess the image
        image = Image.open(image_path).convert('RGB')  # Ensure RGB format
        image = transform(image).unsqueeze(0)  # Apply transform and add batch dimension
        image = image.to(device)  # Move to the same device as the model
        
        # Perform inference
        with torch.no_grad():
            output = model(image)
            probabilities = torch.softmax(output, dim=1)  # Apply softmax to get probabilities
            predicted_idx = torch.argmax(probabilities, dim=1).item()  # Get predicted class index
            predicted_label = classes[predicted_idx]  # Map index to class name
            predicted_prob = probabilities[0, predicted_idx].item()  # Get probability of predicted class
        
        return predicted_label, predicted_prob
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None, None

# Example inference
sample_image_path = 'cats.png'  # Replace with your image path
label, prob = predict_image(sample_image_path, model, val_transform, device, train_dataset.classes)
if label is not None:
    print(f"Predicted: {label}, Probability: {prob:.4f}")